In [19]:
import json
import os
import re
from constants import *

In [21]:
asy_pattern = re.compile(r'\[asy\].*?\[/asy\]', re.DOTALL)
def remove_asy_tags(text):
    return re.sub(asy_pattern, '', text)

In [22]:
def merge_json(
    source_dir, data_class, target_dir, levels: list = [1, 2, 3, 4, 5], test_mode=False
):
    source_dir = os.path.join(source_dir, data_class)
    files = [
        f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))
    ]
    prob_dict = {}
    merged_json = []

    for i in range(len(files) if not test_mode else min(2, len(files))):
        if test_mode:
            print(f"Processing file {i+1}/{len(files)}")
            print(f"working on file path: {os.path.join(source_dir, files[i])}")
        with open(os.path.join(source_dir, files[i]), "r") as f:
            try: # error would occur in reading some files in the geometry class
                data = json.load(f)
                level = int(data["level"][-1])
                if level not in levels:
                    continue
                prob_dict["problem"] = data["problem"]
                prob_dict["solution"] = remove_asy_tags(data["solution"])
                prob_dict["level"] = level
                prob_dict["class"] = data_class
                prob_dict["filename"] = files[i].split(".")[0]
            except:
                print(f"Error in reading file: {files[i]}")
                continue

            merged_json.append(prob_dict.copy())
            
    os.makedirs(os.path.join(target_dir, data_class), exist_ok=True)
    with open(os.path.join(target_dir, data_class, "merged.json"), "w") as f:
        json.dump(merged_json, f, indent=4)

In [28]:
source_dir = "D:/Users/abc/Desktop/MATH/train"
target_dir = 'C:/Users/abc/Dropbox (GaTech)/python_projects/alphaMath/merged_dataset/train'

for data_class in ALL_PROBLEM_CLASSES:
    print('merging class:', data_class)
    merge_json(source_dir, data_class, target_dir, levels=[1,2,3,4,5])